In [1]:
# Import packages
import time
import itertools
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

/Users/ignat/Documents/ehv_scraper/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Start a web driver and open a window
driver = webdriver.Chrome()

# Go to the page with the BI dashboard
driver.get("https://www.hud.gov/program_offices/public_indian_housing/ehv/dashboard")

# Find the iframe that contains the dashboard and switch to it
iframe = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe)

In [4]:
# Click on next page to get to page 2
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next Page"]'))).click()

In [5]:
# Click on next page to get to page 3
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next Page"]'))).click()
# Click on the button to get more data
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fill.ui-role-button-fill.sub-selectable'))).click()

In [6]:
# Grab the column headers
col_headers = driver.find_elements(By.XPATH, '//div[@role="columnheader"]')
col_headers = [x.text for x in col_headers if x.text != 'Row Selection']
col_headers

['PHA Code',
 'PHA Formal Name',
 'Unique ID',
 'Issuance Effective Date',
 'Leased Effective Date',
 'Expired Effective Date',
 'Exit Effective Date\n ',
 'Days since Issuance',
 'Time to Success',
 'Race']

In [7]:
# Define a function to grab and group the data into a nested list
def getSplitTextRows(row):
  cells = row.find_elements(By.XPATH, '//div[@role="gridcell"]')
  text_from_cells = [cell.text for cell in cells] 
  return([list(group) for k, group in itertools.groupby(text_from_cells, lambda x: x=='Select Row') if not k])

In [8]:
# Main scraper loop 
# 1. Scrap the table
# 2. Count rows (X)
# 3. Move down X rows
# 4. Scrap and repeat

# Scrap
stacked_table = pd.DataFrame()
cnt_rows = 0
cnt_new_rows = 1

while cnt_new_rows > 0: 
    current_row = driver.find_element(By.XPATH, '//div[@role="columnheader"]')
    current_data = pd.DataFrame(getSplitTextRows(current_row), columns = col_headers)
    
    # Double click on the last cell available
    cells_in_view = current_row.find_elements(By.XPATH, '//div[@role="gridcell"]')
    first_cell_last_row = cells_in_view[-10]
    ActionChains(driver)\
      .double_click(first_cell_last_row)\
      .perform()
     
    # Scroll down
    key_to_press = "\ue015"
    actions = ActionChains(driver)
    
    # Perform the key press action multiple times with a delay between each press
    for _ in range(len(current_data)):
        actions.key_down(key_to_press).perform()
        time.sleep(0.01)
        
    actions.perform()

    # Count number of new rows
    if len(stacked_table) == 0:
        cnt_new_rows = len(current_data)
    else: 
        cnt_new_rows = (~current_data['Unique ID'].isin(stacked_table['Unique ID'])).sum()

    # Combine scrapped data and display rows scrapped
    stacked_table = pd.concat([stacked_table, current_data], ignore_index=True)
    cnt_rows = len(stacked_table)
    print(f"Processed {cnt_rows} rows.")


Processed 29 rows.
Processed 59 rows.
Processed 86 rows.
Processed 114 rows.
Processed 141 rows.
Processed 164 rows.
Processed 187 rows.
Processed 211 rows.
Processed 241 rows.
Processed 265 rows.
Processed 289 rows.
Processed 312 rows.
Processed 335 rows.
Processed 358 rows.


In [ ]:
stacked_table.drop_duplicates().to_excel('output.xlsx', index=False)